<div style="background:#5D6D7E;padding:20px;color:#ffffff;margin-top:10px;">

# NLP - Práctica 1 ( Extracción de Información) 

## Profesora: Lisibonny Beato
### Período 3-2024-2025</div>


In [8]:
# Importando algunas librerias

import pandas as pd
import re
import spacy
import textacy
import glob
import visualise_spacy_tree
from spacy.matcher import Matcher 
import os
import sys
import en_core_web_trf
import spacy_transformers
import re
import nltk

#añadidas por el grupo
import requests
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
#pip install pandas

In [10]:
#pip install spacy

In [11]:
#pip install textacy

In [12]:
#pip install visualise-spacy-tree

In [13]:
#pip install en-core-web-trf

In [14]:
#pip install spacy-transformers

In [15]:
#pip install nltk

In [16]:
#pip install --upgrade numpy

In [17]:
#pip install --upgrade h5py

In [18]:
#!python -m spacy download en_core_web_trf

<div style="background:#ff6242;padding:20px;color:#ffffff;margin-top:10px;">
<b>Para esta práctica estarán utilizando el corpus MIT Restaurant Corpus, el cual encontrará en este enlace: <a href='https://sls.csail.mit.edu/downloads/restaurant/'>https://sls.csail.mit.edu/downloads/restaurant/</a>.<br /><br />Tome en cuenta que los archivos de train y test están en el formato bio.
<br />
<br />
Aparte del código, debe proveer una interpretación para cada tarea y un análisis para cada resultado obtenido que así lo amerite.</b>
</div>


In [26]:

# Descargar archivo de entrenamiento
url_train = "https://sls.csail.mit.edu/downloads/restaurant/restauranttrain.bio"
response_train = requests.get(url_train)
with open("restauranttrain.bio", "wb") as f:
    f.write(response_train.content)

# Descargar archivo de prueba
url_test = "https://sls.csail.mit.edu/downloads/restaurant/restauranttest.bio"
response_test = requests.get(url_test)
with open("restauranttest.bio", "wb") as f:
    f.write(response_test.content)


## 1. Nivel Básico 
### Puntuación máxima de la tarea: 5 puntos
#### Limpieza y preparación de los datos, Extracción de palabras clave (KPE),  y reconocimiento de entidades relevantes (NER) con modelos heurísticos y pre-entrenados para  reconocer al menos 3 tipos de entidades.

### Extraccion y limpiueza de los datos

In [27]:
def load_bio_data(filepath):
    sentences = []
    sentence = []
    with open(filepath,'r',encoding="utf-8") as file:
        for line in file:
            if line.strip() == "":
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            else:
                parts = line.strip().split()
                if len(parts)>=2:
                    word, tag = parts[0], parts[-1]
                    sentence.append((word, tag))
        if sentence:
            sentences.append(sentence)
    return sentences

In [28]:
train_data = load_bio_data("restauranttrain.bio")
test_data = load_bio_data("restauranttest.bio")

In [29]:
train_sentences = [" ".join([w for w, t in sent]) for sent in train_data]


In [30]:
vectorizer = TfidfVectorizer(stop_words='english',max_features=20)
X = vectorizer.fit_transform(train_sentences)
keywords = vectorizer.get_feature_names_out()

In [31]:
print("Palabras clves:")
print(keywords)

Palabras clves:
['amenity' 'cuisine' 'dish' 'hours' 'location' 'price' 'rating'
 'restaurant_name']


<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Utilice esta celda para colocar comentarios en el notebook, cuando lo estime necesario. Copiela varias veces donde considere.
</div>

## 2. Nivel Intermedio 
### Puntuación máxima de la tarea: 2 puntos
#### Entrenar un modelo propio de NER mediante el pipeline de Spacy

## 3. Nivel Avanzado
### Puntuación máxima de la tarea: 3 puntos
#### Entrenar un modelo de NER mediante el uso de Conditional Random Fields. Recuerde ir más allá que donde llega el ejemplo al respecto provisto en clases.